In [1]:
!pip install python-telegram-bot --upgrade --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.1/717.1 kB 8.2 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/openai/whisper.git -q


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.8 MB/s eta 0:00:00


In [3]:
!apt-get install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
import whisper

# ✅ Load the model once, globally
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 55.3MiB/s]


In [8]:
!pip install gTTS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [5]:
!pip install python-dotenv


# New section

In [ ]:
import os
import tempfile
import requests
from gtts import gTTS
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, MessageHandler, filters
import subprocess
import whisper

# Load the Whisper model once globally
model = whisper.load_model("small")

# Get reply from OpenRouter AI
def get_humanlike_reply(message, lang="hi"):
    api_key = "YOUR_KEY"
    if not api_key:
        return "❌ API key missing!"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    system_prompt = f"""
You are a kind, multilingual AI assistant who helps Indian citizens understand and access government schemes and social services.
Always respond like a human: natural, helpful, empathetic, and friendly.
Always respond in native script of the language (Hindi = देवनागरी, Bengali = বাংলা, Tamil = தமிழ்).
Also ask a polite follow-up if more info is needed.

Language: {lang}
"""

    payload = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
        ]
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"❌ Error: {response.status_code}"

# Convert text to speech and return path to file
def speak_reply(text, lang_code='hi'):
    tts = gTTS(text=text, lang=lang_code)
    path = "reply.mp3"
    tts.save(path)
    return path

# Telegram handler for voice messages
async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    file = await context.bot.get_file(update.message.voice.file_id)
    await update.message.reply_text("🔄 Audio received. Please wait while I process and respond...")

    ogg_path = tempfile.mktemp(suffix=".ogg")
    wav_path = tempfile.mktemp(suffix=".wav")

    await file.download_to_drive(ogg_path)

    # Convert OGG to WAV
    subprocess.run(["ffmpeg", "-i", ogg_path, wav_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Transcribe
    result = model.transcribe(wav_path, task="transcribe")
    latest_transcript = result["text"]

    # Get AI reply
    lang_code = 'hi'  # Change as needed
    reply = get_humanlike_reply(latest_transcript, lang=lang_code)

    # Convert reply to speech
    reply_audio_path = speak_reply(reply, lang_code=lang_code)

    # Send back voice message
    with open(reply_audio_path, 'rb') as audio_file:
        await context.bot.send_voice(chat_id=update.effective_chat.id, voice=audio_file)

# Main entry point
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv

load_dotenv()

TELEGRAM_BOT_TOKEN = "BOT_TOKEN"
if not TELEGRAM_BOT_TOKEN:
    print("❌ Telegram bot token not set.")
else:
    app = ApplicationBuilder().token(TELEGRAM_BOT_TOKEN).build()
    app.add_handler(MessageHandler(filters.VOICE, handle_voice))
    print("✅ Bot is running...")
await app.run_polling()


✅ Bot is running...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
